# DICOM to BIDS converter

Welcome to this DICOM to BIDS converter!

This converter only handles the following MRI sequences:
    - MPRAGE
    - FLAIR
    - EPI
    - Phase
    - Diffusion weighted imaging

The converter should be used on dicom files coming directly out of the MR machine.

#### /!\ Important /!\ : Run steps 1 and 2 before running anything else

Author: Maxence Wynen -- maxencewynen@gmail.com

## 1. Import dependencies

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from dicom2bids import *
from change_orientation import *
print("[INFO] Imports done")
#"C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/WSBIM2243/database"
# Path to dcm2niix.exe converter -- Only for windows(Comment for unix/macOS users) 
dicom2niix_path = "dcm2niix" 
# dicom2niix_path = "C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/dcm2niix.exe" # Uncomment for Windows

[INFO] Imports done


## 2. Set paths

In [4]:
# Output BIDS directory. Should previously be created.
bids_dir = "/home/maggi/Downloads/BIDS_TEMPLATE/"


## 3. Automatically import a DICOM directory into a BIDS format

In [6]:
# Path to dicom directory
directory = "/media/maggi/Elements/DISSECT_MS_DATABASE/sub-006/ses-01/anat/DICOM"
# Convert all DICOMs
dicom_series = convert_all_dicoms(directory, dicom2niix_path)

# Create directories in the BIDS file structure by giving an incremental id
# pat_id, session = make_directories(bids_dir,pat_id=None,session=None)
# To specify the patient id:
pat_id, session = make_directories(bids_dir,pat_id='200',session='01')
# To specify the patient id and session:
# pat_id, session = make_directories(bids_dir,pat_id='ID_TO_SPECIFY',session='SESSION_TO_SPECIFY')

# Rename and move all (interesting) converted files into the bids directory
rename_and_move_nifti(dicom_series, bids_dir, pat_id, session)

print("[INFO] Done")

[INFO] Starting to convert ...
SUBDIR: /media/maggi/Elements/DISSECT_MS_DATABASE/sub-006/ses-01/anat/DICOM/DICOM/Brussel_-_Hopital_Erasme_-_3971583901/DICOM	DIRS: []
SUBDIR: /media/maggi/Elements/DISSECT_MS_DATABASE/sub-006/ses-01/anat/DICOM/EPI	DIRS: []
[INFO] Converted all dicom files to compressed nifti
None
DICOM series not recognized: TOF4CDI0.8mmS3
Path: /media/maggi/Elements/DISSECT_MS_DATABASE/sub-006/ses-01/anat/DICOM/DICOM/Brussel_-_Hopital_Erasme_-_3971583901/DICOM
SERIES: TOF4CDI0.8mmS3
   Filenames: ['_DICOM_AXIALES_FLAIR_20190129105856_802_', '_DICOM_CORO_FLAIR_20190129105856_803_', '_DICOM_CORO_FLAIR_20190129105856_804_', '_DICOM_dADC_epi_20190129105856_402_', '_DICOM_MIP_WIP_TOF4CDI0.8mmS3_SENSE_20190129105856_502__i00001', '_DICOM_MIP_WIP_TOF4CDI0.8mmS3_SENSE_20190129105856_502__i00002', '_DICOM_MIP_WIP_TOF4CDI0.8mmS3_SENSE_20190129105856_502__i00003', '_DICOM_VWIP_Advanced_sweep_SENSE_20190129105856_602_', '_DICOM_VWIP_Advanced_sweep_SENSE_20190129105856_603_', '_DICO

In [5]:
delete_nii_json_in_dicomdir(dicom_series)

## 4.Optional functionalities

### 4.1. Delete subject

In [11]:
# Remove a patient from the database
delete_subject(bids_dir, "2097")

### 4.2. Delete session

In [5]:
# Remove a particular session from a particular patient
delete_session(bids_dir, "026", "03")
print(bids_dir)

/media/maggi/Elements/BACKUP/DISSECT_MS_DATABASE


### 4.3 Rename subject

In [10]:
# Rename a subject in the database
rename_subject(bids_dir, old_id='1000',new_id='2000')

FileNotFoundError: Subject 1000 is not in the database.

### 4.4. Reorient an image

In [5]:
EPI_magnitude = 'acq-mag_T2star'
EPI_phase = 'acq-phase_T2star'
phase_wrapped = 'phase_WRAPPED'
T2 = 'T2'
T1 = 'T1'
T1_Gd = 'T1w_Gd'
FLAIRstar = 'acq-star_FLAIR'
QSM = 'QSM'

reorient(subject = '026', session = '02', 
         sequence_name = QSM + '_matlab', 
         axis = 'z',
         DIR=bids_dir)

In [34]:
import shutil
for sub in range(26,44):
    sub = str(sub).zfill(3)
    for ses in range(1,6):
        ses = str(ses).zfill(2)
        if not os.path.exists(os.path.join(bids_dir, f'sub-{sub}', f'ses-{ses}', 'anat', f'sub-{sub}_ses-{ses}_QSM_orig.nii.gz')):
            continue
        print(sub, ses)
        """
        root = os.path.join(bids_dir, f'sub-{sub}', f'ses-{ses}', 'anat') 
        os.remove(os.path.join(root, f'sub-{sub}_ses-{ses}_QSM_orig.nii.gz'))
        os.remove(os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_magnitude + '_orig.nii.gz'))
        os.remove(os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_phase + '_orig.nii.gz'))
        
        shutil.move(os.path.join(root, f'sub-{sub}_ses-{ses}_QSM_orig_z.nii.gz'), 
                    os.path.join(root, f'sub-{sub}_ses-{ses}_QSM_orig.nii.gz'))
        shutil.move(os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_magnitude + '_orig_z.nii.gz'),
                   os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_magnitude + '_orig.nii.gz'))
        shutil.move(os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_phase + '_orig_z.nii.gz'),
                   os.path.join(root, f'sub-{sub}_ses-{ses}_'+ EPI_phase + '_orig.nii.gz'))
        
        
        reorient(subject = sub, session = ses, 
                 sequence_name = QSM + '_orig', 
                 axis = 'z',
                 DIR=bids_dir)
        reorient(subject = sub, session = ses, 
                 sequence_name = EPI_magnitude + '_orig', 
                 axis = 'z',
                 DIR=bids_dir)
        reorient(subject = sub, session = ses, 
                 sequence_name = EPI_phase + '_orig', 
                 axis = 'z',
                 DIR=bids_dir)
        """

026 01
026 02
027 01
027 02
028 01
028 02
029 01
029 02
030 01
030 02
031 01
031 02
032 01
033 01
033 02
034 01
034 02
035 01
035 02
035 03
036 01
036 02
036 03
037 01
037 02
038 02
039 01
040 01
040 02
041 01
041 02
042 01
042 02
043 01
043 02
